#  Extração de Dados com Python 🐍

Estou desenvolvendo o Healtscore da empresa em Power BI, um dos pilares principais é a pesquisa de satisfação dos nossos clientes e colaboradores, utilizamos o Freshserivice que gerencia as ocorrências de tickets. 

[O Freshservice fornece uma API](https://api.freshservice.com/#intro), onde conseguimos consultar todos os tickets de uma só vez, felicitando a extração com um sistema paginado de no máximo 100 tickets por consulta. 
A pesquisa de satisfação e as atividades desenvolvidas em cada tickets, não temos um endpoint que consulta diversos tickets de uma só vez.

## Solução
Desenvolvi um script que lê nassa tabela de tickets já gerada e salva em csv, em seguida realiza um loop rodando para cada ticket uma consulta na API respeitando os rate limite, cria uma coluna onde se armazena o ID do ticket utilizado para fazer a consulta em cada linha da resposta e por fim salva em arquivos csv cada tabela gerada.


In [ ]:
import pandas as pd
import requests as req
import time
import json


In [ ]:

#Realiza a leitura do arquivo csv
TabelaTickets = pd.read_csv("tickets.csv", delimiter="|")
token = "XXXXXXXXXXXXXXXXXXXXXXXXX"

#Funções para facilitar e diminu o codigo no loop 
def csat(ticket): 
    return f"https://grupometarh.freshservice.com/api/v2/tickets/{ticket}/csat_response"

def activities(ticket):
    return f"https://grupometarh.freshservice.com/api/v2/tickets/{ticket}/activities"

TabelaCsat = []
TabelaActivities = []
x = 1

for ticket in TabelaTickets["id"]:

    time.sleep(1)
    x = x + 1 
    resultadoc = 0
    resultadoa = 0
    
    #Realiza a consulta nos endpoint 
    Respostacsat = req.get(csat(ticket),headers={"Authorization": token})
    RespostaActi = req.get(activities(ticket),headers={"Authorization": token})

    if Respostacsat.status_code == 200:
       resultadoc = Respostacsat.json().get("csat_response",[]) #retorno da API para cada ticket 
       resultadoc['Ticket ID'] = ticket #armazena o numero do ticket para cada pesquisq 
       if resultadoc is not None:
            TabelaCsat.append(resultadoc)  #Armazena o resultado puxado em uma lista final para todas as buasca
       
    
    if RespostaActi.status_code ==  200:
        resultadoa = RespostaActi.json().get("activities",[]) #Retorna as linhas de todoas atividades de cada tickets
        Linhas = len(resultadoa)#Armazenando a quantidade de linhas retornada 
        FinalLista = len(TabelaActivities) #Busco a quantidade de linha na lista existente 
           
        linha = 0
        for linha in range(Linhas): 
            resultadoa[linha]['Ticket ID'] = ticket #Entra em cada linha retornada e armazena o numero do ticket

               
        if resultadoa is not None:
            TabelaActivities.extend(resultadoa) #Armazena o resultado puxado em uma lista final para todas as buasca
    
    DFa = pd.DataFrame(TabelaActivities)
    DFa.to_csv("attivities.csv")
    DFc = pd.DataFrame(TabelaCsat)
    DFc.to_csv("csat.csv")
   
 
    print(x)



